<a href="https://colab.research.google.com/github/RMottram/2D-CNN/blob/master/Network_signals_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Oct  8 15:42:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# !unzip /content/drive/MyDrive/Colab\ Notebooks/64x64.zip > /dev/null
# !unzip /content/drive/MyDrive/Colab\ Notebooks/32x32.zip > /dev/null
# !unzip /content/drive/MyDrive/Colab\ Notebooks/split_images.zip > /dev/null
!unzip /content/drive/MyDrive/Colab\ Notebooks/datasets_V2.zip > /dev/null

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imblearn, sys, os, time

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras
# from keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

# Settings
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
## 80/20
normal_train = pd.read_csv('/content/v2/normal_train_8020.csv')
normal_test = pd.read_csv('/content/v2/normal_test_8020.csv')
ip_train = pd.read_csv('/content/v2/ip_train_8020.csv')
ip_test = pd.read_csv('/content/v2/ip_test_8020.csv')
port_train = pd.read_csv('/content/v2/port_train_8020.csv')
port_test = pd.read_csv('/content/v2/port_test_8020.csv')

## 70/30
# normal_train = pd.read_csv('/content/v2/normal_train_7030.csv')
# normal_test = pd.read_csv('/content/v2/normal_test_7030.csv')
# ip_train = pd.read_csv('/content/v2/ip_train_7030.csv')
# ip_test = pd.read_csv('/content/v2/ip_test_7030.csv')
# port_train = pd.read_csv('/content/v2/port_train_7030.csv')
# port_test = pd.read_csv('/content/v2/port_test_7030.csv')

## 60/40
# normal_train = pd.read_csv('/content/v2/normal_train_6040.csv')
# normal_test = pd.read_csv('/content/v2/normal_test_6040.csv')
# ip_train = pd.read_csv('/content/v2/ip_train_6040.csv')
# ip_test = pd.read_csv('/content/v2/ip_test_6040.csv')
# port_train = pd.read_csv('/content/v2/port_train_6040.csv')
# port_test = pd.read_csv('/content/v2/port_test_6040.csv')


In [5]:
normal_train.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,icmp,tcp,udp,IRC,X11,Z39_50,auth,bgp,courier,csnet_ns,ctf,daytime,discard,domain,domain_u,echo,eco_i,ecr_i,efs,exec,finger,ftp,ftp_data,gopher,hostnames,http,http_443,imap4,iso_tsap,klogin,kshell,ldap,link,login,mtp,name,netbios_dgm,netbios_ns,netbios_ssn,netstat,nnsp,nntp,ntp_u,other,pm_dump,pop_2,pop_3,private,remote_job,shell,smtp,sql_net,ssh,sunrpc,supdup,systat,telnet,tftp_u,tim_i,time,urp_i,uucp,uucp_path,vmnet,whois,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH,attack_type
0,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,66.86888,106.79061,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,2.55000,255.00000,254.00000,255.00000,2.55000,2.55000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
1,0.00000,0.00005,0.00013,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,5.98826,6.98630,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,35.70000,68.00000,255.00000,255.00000,0.00000,2.55000,5.10000,0.00000,0.00000,0.00000,2.55000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
2,0.13255,0.00001,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.49706,2.99413,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,84.15000,255.00000,240.00000,239.70000,5.10000,15.30000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
3,0.00000,0.00006,0.00277,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0

In [7]:
## merging the appropriate datasets together
normal_ip_train = normal_train.append(ip_train)
normal_ip_test = normal_test.append(ip_test)

normal_port_train = normal_train.append(port_train)
normal_port_test = normal_test.append(port_test)

In [13]:
normal_ip_train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,icmp,tcp,udp,IRC,X11,Z39_50,auth,bgp,courier,csnet_ns,ctf,daytime,discard,domain,domain_u,echo,eco_i,ecr_i,efs,exec,finger,ftp,ftp_data,gopher,hostnames,http,http_443,imap4,iso_tsap,klogin,kshell,ldap,link,login,mtp,name,netbios_dgm,netbios_ns,netbios_ssn,netstat,nnsp,nntp,ntp_u,other,pm_dump,pop_2,pop_3,private,remote_job,shell,smtp,sql_net,ssh,sunrpc,supdup,systat,telnet,tftp_u,tim_i,time,urp_i,uucp,uucp_path,vmnet,whois,OTH,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH,attack_type
0,0.00000,0.00001,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,66.86888,106.79061,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,2.55000,255.00000,254.00000,255.00000,2.55000,2.55000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
1,0.00000,0.00005,0.00013,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,5.98826,6.98630,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,35.70000,68.00000,255.00000,255.00000,0.00000,2.55000,5.10000,0.00000,0.00000,0.00000,2.55000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
2,0.13255,0.00001,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.49706,2.99413,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,84.15000,255.00000,240.00000,239.70000,5.10000,15.30000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,normal
3,0.00000,0.00006,0.00277,0.00000,0.00000,0.00000,0.00000,0.00000,255.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0

In [8]:
normal_ip_train_class = pd.Series(normal_ip_train.iloc[:,-1])
normal_port_train_class = pd.Series(normal_port_train.iloc[:,-1])

normal_ip_test_class = pd.Series(normal_ip_test.iloc[:,-1])
normal_port_test_class = pd.Series(normal_port_test.iloc[:,-1])

In [9]:
normal_ip_train_class

pd.get_dummies(normal_ip_train_class)

,ipsweep,normal
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
195323,1,0
195324,1,0
195325,1,0
195326,1,0


In [10]:
normal_ip_train = normal_ip_train.drop(['attack_type'], axis = 1)
normal_ip_test = normal_ip_test.drop(['attack_type'], axis = 1)

normal_port_train = normal_port_train.drop(['attack_type'], axis = 1)
normal_port_test = normal_port_test.drop(['attack_type'], axis = 1)

In [11]:
ann = Sequential()

## input layer and hidden layer #1
ann.add(Dense(input_dim = 114, units = 12, kernel_initializer = 'uniform', activation = 'relu'))

## hidden layer #2
ann.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

## hidden layer #3
ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

## output layer
ann.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# callback = [EarlyStopping(monitor='accuracy', mode='max', patience=3 )]

ann.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                1380      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 14        
Total params: 1,590
Trainable params: 1,590
Non-trainable params: 0
_________________________________________________________________


In [12]:
start = time.time()

ann.fit(normal_ip_train, pd.get_dummies(normal_ip_train_class), batch_size = 1024, epochs = 200)

end = time.time() - start

print(f'Training for ANN took {round(end,3)} seconds')

Epoch 1/200
251/251 [==============================] - 4s 5ms/step - loss: 0.1192 - accuracy: 0.9767
Epoch 2/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0028 - accuracy: 0.9991
Epoch 3/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0020 - accuracy: 0.9992
Epoch 4/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0016 - accuracy: 0.9994
Epoch 5/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0014 - accuracy: 0.9995
Epoch 6/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 7/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 8/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 9/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 10/200
251/251 [==============================] - 1s 5ms/step - loss: 0.0011 - accura

In [14]:
loss, acc = ann.evaluate(normal_ip_test, pd.get_dummies(normal_ip_test_class), batch_size=1024, verbose=0)
print(f'Loss = {round(loss, 3)}\tAccuracy = {round(acc, 4)*100}%')

Loss = 0.0	Accuracy = 99.99%


In [22]:
preds = (ann.predict(pd.get_dummies(normal_ip_test)) > 0.5)

In [23]:
preds

array([[False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [Fal

In [16]:
def build_classifier():
    
    ann = Sequential()
    ann.add(Dense(input_dim = 114, units = 12, kernel_initializer = 'uniform', activation = 'relu'))
    ann.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
    ann.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
    ann.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    ann.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'sigmoid'))
    ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return ann

clf = KerasClassifier(build_fn = build_classifier, batch_size = 1024, epochs = 200)

In [24]:
start = time.time()

scores = cross_val_score(clf, X = normal_ip_test, y = pd.get_dummies(normal_ip_test_class), cv = 5, n_jobs = -1)
accuracy = metrics.accuracy_score(pd.get_dummies(normal_ip_test_class), preds)
# confusion_matrix = metrics.confusion_matrix(Y_test, yhat_test)
# classification = metrics.classification_report(normal_ip_test, preds.argmax(axis = 1), target_names=pd.get_dummies(normal_ip_test_class).columns)

end = time.time() - start

print('\n============================== ANN Model Test Results ==============================\n')
print(f'ANN evaluation took {round(end, 3)} seconds\n')
print(f'Model Cross Validation: \n {round(scores.mean(), 3)*100}%')
print('\n')
print (f"Model Accuracy: \n {round(accuracy, 3)*100}%")
print('\n')
# print("Confusion matrix:" "\n", confusion_matrix)
# print('\n')
print("Classification report:" "\n", classification)


============================== ANN Model Test Results ==============================

ANN evaluation took 237.752 seconds

Model Cross Validation: 
 100.0%


Model Accuracy: 
 100.0%




NameError: ignored